To use MongoDB as a toolbox, you will need to complete the following steps:

1. Register for a MongoDB Account:
   - Go to the MongoDB website (https://www.mongodb.com/cloud/atlas/register).
   - Click on the "Try Free" or "Get Started Free" button.
   - Fill out the registration form with your details and create an account.

2. Create a [MongoDB Cluster](https://www.mongodb.com/docs/atlas/tutorial/deploy-free-tier-cluster/#procedure)

3. Set Up [Database Access](https://www.mongodb.com/docs/atlas/security-add-mongodb-users/#add-database-users):
   - In the left sidebar, click on "Database Access" under "Security".
   - Click "Add New Database User".
   - Create a username and a strong password. Save these credentials securely.
   - Set the appropriate permissions for the user (e.g., "Read and write to any database").

4. Configure Network Access:
   - In the left sidebar, click on "Network Access" under "Security".
   - Click "Add IP Address".
   - To allow access from anywhere (not recommended for production), enter 0.0.0.0/0.
   - For better security, whitelist only the specific IP addresses that need access.

5. Obtain the [MongoDB URI](https://www.mongodb.com/docs/manual/reference/connection-string/#find-your-mongodb-atlas-connection-string):

Now that you have your MongoDB URI, you can use it to connect to your cluster in the `memorizz` library.

In [9]:
%pip install memorizz

Note: you may need to restart the kernel to use updated packages.


In [1]:
from memorizz.database.mongodb import MongoDBToolsConfig, MongoDBTools

In [2]:
import os
import getpass

# Set up environment variables for API keys and MongoDB URI
# This key is required for using OpenAI's services, such as generating embeddings
OPENAI_API_KEY = getpass.getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# This URI is needed to connect to the MongoDB database
MONGO_URI = getpass.getpass("Enter MongoDB URI: ")
os.environ["MONGO_URI"] = MONGO_URI

In [3]:
# 1. Initialize the MongoDB configuration and create a MongoDB tools instance
config = MongoDBToolsConfig(
    mongo_uri=MONGO_URI,  # MongoDB connection string
    db_name="function_calling_db",  # Name of the database to use
    collection_name="tools",  # Name of the collection to store tools
    embedding_model="text-embedding-3-small",  # OpenAI model for generating embeddings
    embeding_dimensions_size=1536,  # Dimension size for the embedding vectors
    vector_search_candidates=150,  # Number of candidates to consider in vector search
    vector_index_name="vector_index"  # Name of the vector index in MongoDB
)

# 2. Create an instance of MongoDBTools with the configured settings
mongodb_tools = MongoDBTools(config)

# 3. Create a decorator function for registering tools
mongodb_toolbox = mongodb_tools.mongodb_toolbox

INFO:memorizz.database.mongodb.mongodb_tools:MongoDBTools initialized successfully.


In [4]:
import random
from datetime import datetime

# 4. Define and register tool functions using the mongodb_toolbox decorator
# These functions will be stored in the MongoDB database and can be retrieved for function calling
@mongodb_toolbox()
def shout(statement: str) -> str:
  """
  Convert a statement to uppercase letters to emulate shouting. Use this when a user wants to emphasize something strongly or when they explicitly ask to 'shout' something..

  """
  return statement.upper()

@mongodb_toolbox()
def get_weather(location: str, unit: str = "celsius") -> str:
    """
    Get the current weather for a specified location.
    Use this when a user asks about the weather in a specific place.

    :param location: The name of the city or location to get weather for.
    :param unit: The temperature unit, either 'celsius' or 'fahrenheit'. Defaults to 'celsius'.
    :return: A string describing the current weather.
    """
    conditions = ["sunny", "cloudy", "rainy", "snowy"]
    temperature = random.randint(-10, 35)

    if unit.lower() == "fahrenheit":
        temperature = (temperature * 9/5) + 32

    condition = random.choice(conditions)
    return f"The weather in {location} is currently {condition} with a temperature of {temperature}°{'C' if unit.lower() == 'celsius' else 'F'}."

@mongodb_toolbox()
def get_stock_price(symbol: str) -> str:
    """
    Get the current stock price for a given stock symbol.
    Use this when a user asks about the current price of a specific stock.

    :param symbol: The stock symbol to look up (e.g., 'AAPL' for Apple Inc.).
    :return: A string with the current stock price.
    """
    price = round(random.uniform(10, 1000), 2)
    return f"The current stock price of {symbol} is ${price}."

@mongodb_toolbox()
def get_current_time(timezone: str = "UTC") -> str:
    """
    Get the current time for a specified timezone.
    Use this when a user asks about the current time in a specific timezone.

    :param timezone: The timezone to get the current time for. Defaults to 'UTC'.
    :return: A string with the current time in the specified timezone.
    """
    current_time = datetime.utcnow().strftime("%H:%M:%S")
    return f"The current time in {timezone} is {current_time}."


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:memorizz.database.mongodb.mongodb_tools:Successfully registered tool: shout
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:memorizz.database.mongodb.mongodb_tools:Successfully registered tool: get_weather
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:memorizz.database.mongodb.mongodb_tools:Successfully registered tool: get_stock_price
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:memorizz.database.mongodb.mongodb_tools:Successfully registered tool: get_current_time


In [5]:
# 5. Define the user query
# This query will be used to search for relevant tools in the MongoDB database
user_query = "Hi, can you shout the statement: We are there"

In [6]:
# 6. Populate tools based on the user query
tools = mongodb_tools.populate_tools(
    user_query,  # The query string to search for relevant tools
    num_tools=2  # The maximum number of tools to return from the search
    )

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:memorizz.database.mongodb.mongodb_tools:Successfully populated 2 tools


{'_id': ObjectId('66d45ce461c366b24e31705d'), 'name': 'shout', 'description': "Convert a statement to uppercase letters to emulate shouting. Use this when a user wants to emphasize something strongly or when they explicitly ask to 'shout' something..", 'parameters': {'type': 'object', 'properties': {'statement': {'type': 'string', 'description': 'Parameter statement'}}, 'required': ['statement'], 'additionalProperties': False}}
{'_id': ObjectId('66d45ce561c366b24e31708a'), 'name': 'get_stock_price', 'description': "Get the current stock price for a given stock symbol.\nUse this when a user asks about the current price of a specific stock.\n\n:param symbol: The stock symbol to look up (e.g., 'AAPL' for Apple Inc.).\n:return: A string with the current stock price.", 'parameters': {'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'Parameter symbol'}}, 'required': ['symbol'], 'additionalProperties': False}}


In [7]:
import pprint

pprint.pprint(tools)

[{'function': {'description': 'Convert a statement to uppercase letters to '
                              'emulate shouting. Use this when a user wants to '
                              'emphasize something strongly or when they '
                              "explicitly ask to 'shout' something..",
               'name': 'shout',
               'parameters': {'additionalProperties': False,
                              'properties': {'statement': {'description': 'Parameter '
                                                                          'statement',
                                                           'type': 'string'}},
                              'required': ['statement'],
                              'type': 'object'}},
  'type': 'function'},
 {'function': {'description': 'Get the current stock price for a given stock '
                              'symbol.\n'
                              'Use this when a user asks about the current '
                      